In [29]:
import torch #may take some times
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import random

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_bengali_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_bengali_model")

In [33]:
input_text = "১ম প্রকাশ"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)
print(f"Prediction: {predictions.item()}")

Prediction: 1


In [35]:
!pip install python-docx

In [36]:
!pip install unicodeconverter

In [37]:
!pip install langid

In [38]:
import unicodeconverter as uc
import docx
def unicoded_text(bijoy_text):
  unicode_text = uc.convert_bijoy_to_unicode(bijoy_text)
  return unicode_text

def reverse_unicode(unicode_text):
    bijoy_text = uc.convert_unicode_to_bijoy(unicode_text)
    return bijoy_text

In [40]:
#Reading The Doc File Using Python Docx Library
doc_file_path = r'C:\Users\minha\Downloads\Project eBook Automation\Project eBook Automation\Ebook\416455.docx' #you have to pass your ebook location
original_doc = docx.Document(doc_file_path)

In [41]:
header = original_doc.paragraphs[:500]

In [47]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter
from docx import Document
import unicodedata
import re
from docx.shared import Pt

pdfmetrics.registerFont(TTFont('SutonnyMJ', r'C:\Users\minha\Downloads\sutonnymj-regular\SutonnyMJ Regular\SutonnyMJ Regular.ttf'))
pdfmetrics.registerFont(TTFont('AlMajid-Quranic', r'C:\Users\minha\Downloads\al-majeed-quranic-font-regular\Al Majeed Quranic Font Regular\Al Majeed Quranic Font Regular.ttf'))

def get_prediction(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    if predictions.item() == 0:
        return "Unnecessary Text"
    elif predictions.item() == 2:
        return "Index"
    else:
        return "Necessary Text"


def is_url(word):
    url_pattern = r'\b(?:https?|ftp)?(?::\/\/)?(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?:\/[^\s]*)?\b'
    return bool(re.match(url_pattern, word))


body_index = 0
header_paragraphs=[]
for index,paragraph in enumerate(header):
    text = paragraph.text
    converted_text = unicoded_text(text)
    prediction = get_prediction(converted_text)
    if converted_text in ['সূচি','সূচিপত্র',"index"]:
        body_index = index 
        break
    if prediction == "Unnecessary Text" or is_url(converted_text):
        continue
    else:
        font_weight = 'bold' if any(run.bold for run in paragraph.runs) else 'normal'
        font_size = next((run.font.size for run in paragraph.runs if run.font.size), None)
        if font_size:
            font_size = Pt(font_size / 12700).pt
        header_paragraphs.append({
            'text': text,
            'bold': font_weight == 'bold',
            'font_size': font_size
        })
        
after_duplicate_removal = []
seen = set()
for d in header_paragraphs:
    dict_tuple = tuple(sorted(d.items()))
    if dict_tuple not in seen:
        seen.add(dict_tuple)
        after_duplicate_removal.append(d)
after_duplicate_removal


[{'text': '', 'bold': True, 'font_size': 12.5},
 {'text': '', 'bold': False, 'font_size': None},
 {'text': '', 'bold': False, 'font_size': 12.5},
 {'text': 'হেলেন হ্যানফ্', 'bold': True, 'font_size': 14.0},
 {'text': 'ভাষান্তর: ', 'bold': False, 'font_size': 12.0},
 {'text': 'তানজীম', 'bold': True, 'font_size': 12.0},
 {'text': 'ফারহানা জাহান', 'bold': True, 'font_size': 12.0},
 {'text': ' ', 'bold': False, 'font_size': 12.0},
 {'text': 'অনুবাদকের উৎসর্গ ', 'bold': True, 'font_size': 18.0},
 {'text': 'ফারহানা জাহান', 'bold': False, 'font_size': 13.0},
 {'text': 'তানজীম', 'bold': False, 'font_size': 13.0},
 {'text': 'মুখবন্ধ', 'bold': True, 'font_size': 20.0},
 {'text': 'অ্যান ব্যানক্রফট', 'bold': True, 'font_size': 13.0},
 {'text': ' অক্টোবর ৫, ১৯৪৯', 'bold': False, 'font_size': 13.0},
 {'text': 'ইংল্যান্ড।', 'bold': False, 'font_size': 13.0},
 {'text': 'মহোদয়বৃন্দ, ', 'bold': False, 'font_size': 13.0},
 {'text': 'বিনীতা', 'bold': False, 'font_size': 13.0},
 {'text': 'হেলেন হ্যানফ্', '

In [48]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION
from docx.enum.section import WD_ORIENT
from docx.shared import Pt, Inches
from docx.enum.section import WD_SECTION_START
from docx.oxml import OxmlElement
import langid
from docx.oxml.ns import qn

doc = Document()
sections = doc.sections
for section in sections:
    section.top_margin = Inches(1)
    section.bottom_margin = Inches(1)
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

    section.vertical_alignment =WD_PARAGRAPH_ALIGNMENT.CENTER

for item in after_duplicate_removal:
    paragraph = doc.add_paragraph()
    run = paragraph.add_run(item['text'])
    run.bold = item['bold']
    
    if item['font_size']:
        run.font.size = Pt(item['font_size'])
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

body_section = original_doc.paragraphs[body_index:]
body_paragraphs=[]

for index,paragraph in enumerate(body_section):
    text = paragraph.text
    if is_url(text):
        continue
    font_weight = 'bold' if any(run.bold for run in paragraph.runs) else 'normal'
    font_size = next((run.font.size for run in paragraph.runs if run.font.size), None)
    if font_size:
        font_size = Pt(font_size / 12700).pt
    body_paragraphs.append({
        'text': text,
        'bold': font_weight == 'bold',
        'font_size': font_size
    })

for item in body_paragraphs:
    paragraph = doc.add_paragraph()
    run = paragraph.add_run(item['text'])
    
    if item['font_size']:
        run.font.size = Pt(item['font_size'])
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

def set_font(run, font_name='SutonnyMJ'):
    run.font.name = font_name
    r = run._r
    rPr = r.get_or_add_rPr()
    rFonts = OxmlElement('w:rFonts')
    rFonts.set(qn('w:ascii'), font_name)
    rFonts.set(qn('w:eastAsia'), font_name)
    rPr.append(rFonts)

for paragraph in doc.paragraphs:
    text = unicoded_text(paragraph.text)
    lang, confidence = langid.classify(text)
    if lang == "bn" or lang =="as":
        for run in paragraph.runs:
            set_font(run, 'SutonnyMJ')
    if lang == "en":
        for run in paragraph.runs:
            set_font(run, 'Times New Roman')
    if lang == "ar" or lang == "fs":
        for run in paragraph.runs:
            set_font(run, 'Al Majeed Quranic')

doc.save(r'E:\Corrected Text 2.docx')
